# Set Up

Run the following cell to load your data and some utility functions.

In [1]:
import pandas as pd
import numpy as np
from undyingkingdoms.static.metadata import *

Attempt to import races and write data to .xlsx

In [48]:
races = pd.DataFrame({"Race": metadata.metadata_races})
print(races)
# races.to_excel('undyingkingdoms/static/metadata/all.xlsx', sheet_name='Race Table')

     Race
0   Human
1     Elf
2   Dwarf
3  Goblin


Attempt to add modifiers to races.

In [5]:
modifiers = [mod[:-9] for mod in dir(metadata) if "modifier" in mod]
modifiers = pd.DataFrame({"Modifier": modifiers})
# print(modifiers)

foo = {races.columns[0]: [], modifiers.columns[0]: []}
for race in races.Race:
    for mod in modifiers.Modifier:
        foo[races.columns[0]].append(race)
        foo[modifiers.columns[0]].append(mod)

race_mods = pd.DataFrame(foo)
race_mods.to_excel('undyingkingdoms/static/metadata/all.xlsx', sheet_name='Race Modifiers')

Attempt to pull modifier values from metadata.

In [2]:
modifiers = [mod for mod in dir(metadata) if "modifier" in mod]
mod_dfs = {' '.join(mod.split('_')).title(): pd.DataFrame(getattr(metadata, mod)) for mod in modifiers}

mod_types = [' '.join(mod.split('_')).title() for mod in modifiers]
cols = ["Race", "Background", "Modifier Label", "Modifier Type", "Modifier Value", "Modifier Description"]
# cols = ['Races', 'BackgroundsModifier Names', 'Amount Of Thieves Modifier', 'Birth Rate Modifier', 'Death Rate Modifier', 'Defense Per Citizen Modifier', 'Food Consumed Modifier', 'Happiness Modifier', 'Income Modifier', 'Infiltration Success Modifier', 'Offensive Power Modifier', 'Production Per Worker Modifier']
df = pd.DataFrame(index=range(44), columns=cols)

i = 0
# print(cols)
for mod_key, mod_df in mod_dfs.items():
    for r_b in mod_df:
        if r_b in metadata.metadata_races:
            df.Race[i] = r_b
        else:  # col is Background
            df.Background[i] = r_b
        df['Modifier Type'][i] = mod_key
        df['Modifier Label'][i] = mod_df[r_b][0]
        df['Modifier Value'][i] = mod_df[r_b][1]
        i += 1

for race in metadata.metadata_races:
    if race not in df.Race.dropna().unique():
#         print(race)
        df.Race[i] = race
        i += 1

for background in metadata.metadata_backgrounds:
    if background not in df.Background.dropna().unique():
        print(background)
        df.Background[i] = background
        i += 1
df = df.sort_values(['Race', 'Background', 'Modifier Label']).reset_index(drop=True)


for name in df['Modifier Label']:
    try:
        row = df['Modifier Label'] == name
        val = df.loc[row, "Modifier Value"].iloc[0]
        if int(val) == val:  # is a fixed value
            if val > 0:
                val = '+' + str(int(val))
        else:  # is a percent
            if val > 0:
                val = '+' + str(int(val * 100))
            val = str(val) + '%'
        label = df.loc[row, "Modifier Type"].iloc[0][:-9]
        df.loc[row, 'Modifier Description'] = '{}: {} {}'.format(name, val, label)
    except IndexError:
        pass

df.to_excel('undyingkingdoms/static/metadata/modifiers.xlsx', sheet_name='Modifiers')
df

,Race,Background,Modifier Label,Modifier Type,Modifier Value,Modifier Description
0,Dwarf,NaN,Dwarven Steel,Production Per Worker Modifier,0.15,Dwarven Steel: +15% Production Per Worker
1,Dwarf,NaN,Ravenous,Food Consumed Modifier,0.15,Ravenous: +15% Food Consumed
2,Elf,NaN,Citizen Militia,Defense Per Citizen Modifier,1,Citizen Militia: +1 Defense Per Citizen
3,Elf,NaN,Elders,Birth Rate Modifier,-0.15,Elders: -0.15% Birth Rate
4,Goblin,NaN,Expendable,Birth Rate Modifier,0.15,Expendable: +15% Birth Rate
5,Goblin,NaN,Infighting,Happiness Modifier,-1,Infighting: -1 Happiness
6,Goblin,NaN,Sneaky,Infiltration Success Modifier,0.1,Sneaky: +10% Infiltration Success
7,Human,NaN,NaN,NaN,NaN,NaN
8,NaN,Engineer,Artisan,Production Per Worker Modifier,0.2,Artisan: +20% Production Per Worker
9,NaN,Engineer,Craftsman,Buildings Built Per Day Modifier,1,Craftsman: +1 Buildings Built Per Day


Try and work out what data to pull into the Guide Race Compare page.

In [6]:
df = pd.read_excel('undyingkingdoms/static/metadata/modifiers.xlsx', sheet_name='Modifiers', index_col=0)

for race in df.Race.unique():
    select = df['Race'] == race
    for row in df.loc[select, 'Modifier Description']:
        if row:
            print(row)

Dwarven Steel: +15% Production Per Worker
Ravenous: +15% Food Consumed
Citizen Militia: +1 Defense Per Citizen
Elders: -0.15% Birth Rate
Expendable: +15% Birth Rate
Infighting: -1 Happiness
Sneaky: +10% Infiltration Success
nan


Attempt to pull soldier data out of metadata files.

In [3]:
gen_armies = metadata_armies_all.generic_armies
army_args = ['base_name', 'class_name', 'class_name_plural', 'total', 'trainable_per_day',
 'gold', 'iron', 'wood', 'upkeep', 'attack', 'defence', 'health', 'description']

cols = ['Race', 'Base Name', 'Class Name', 'Class Name Plural', 'Total',
        'Trainable Per Day', 'Gold', 'Iron', 'Wood', 'Upkeep', 'Attack',
        'Defence', 'Health', 'Description']
size = len(gen_armies) * len(metadata.metadata_races)
df = pd.DataFrame(index=range(size), columns=cols)

i = 0
for k, v in gen_armies.items():
    for j, col in enumerate(cols[1:]):
        df.loc[i, col] = getattr(v, army_args[j])
    i += 1

race_armies = [metadata_armies_human, metadata_armies_elf, metadata_armies_dwarf, metadata_armies_goblin]

for race in metadata.metadata_races:
    mod = locals()['metadata_armies_' + race.lower()]
    army = getattr(mod, race.lower() + '_armies')
    for k, v in army.items():
        df.loc[i, 'Race'] = race
        for j, col in enumerate(cols[1:]):
            df.loc[i, col] = getattr(v, army_args[j])
        i += 1

df.to_excel('undyingkingdoms/static/metadata/armies.xlsx', sheet_name='Armies')
# gen_armies
# cols
df

,Race,Base Name,Class Name,Class Name Plural,Total,Trainable Per Day,Gold,Iron,Wood,Upkeep,Attack,Defence,Health,Description
0,NaN,peasant,peasant,peasants,25,25,5,2,1,5,1,1,1,Unknown
1,NaN,soldier,soldier,soldiers,0,15,15,6,3,15,3,1,3,Unknown
2,NaN,archer,archer,archers,30,10,20,8,4,15,0,5,3,Unknown
3,NaN,elite,elite,elites,0,5,40,15,6,25,8,5,6,Unknown
4,NaN,monster,monster,monsters,0,1,250,75,50,100,50,50,25,Unknown
5,Human,peasant,man-at-arms,men-at-arms,25,50,4,1,1,5,1,1,1,Men-at-arms can be trained extremely quickly a...
6,Human,soldier,footman,footmen,0,20,15,5,3,15,3,1,3,Footmen are strong attackers.
7,Human,archer,musketeer,musketeers,30,15,20,7,4,15,0,5,3,Crossbowmen are excellent at defending.
8,Human,elite,knight,knights,0,8,40,14,6,25,8,5,6,Knights are one of the best attackers.
9,Human,monster,gryphon,gryphons,0,1,250,75,50,100,50,50,25,Gryphons are incredibly powerful monsters.


Work out how to display troop data.

In [8]:
armies = pd.read_excel('undyingkingdoms/static/metadata/armies.xlsx', sheet_name='Armies', index_col=0)
# print(df)
for race in metadata.metadata_races:
    
    select = armies.Race == race
    army = armies.loc[select].reset_index()
    for i in range(len(army)):
        print(army.loc[i, 'Class Name'].title())
        print(army.Gold[i])
#     print(armies.loc[select, "Class Name Plural"])
#     for army in armies[select]:
#         print(army)
    break
armies
army

Man-At-Arms
4
Footman
15
Musketeer
20
Knight
40
Gryphon
250


,index,Race,Base Name,Class Name,Class Name Plural,Total,Trainable Per Day,Gold,Iron,Wood,Upkeep,Attack,Defence,Health,Description
0,5,Human,peasant,man-at-arms,men-at-arms,25,50,4,1,1,5,1,1,1,Men-at-arms can be trained extremely quickly a...
1,6,Human,soldier,footman,footmen,0,20,15,5,3,15,3,1,3,Footmen are strong attackers.
2,7,Human,archer,musketeer,musketeers,30,15,20,7,4,15,0,5,3,Crossbowmen are excellent at defending.
3,8,Human,elite,knight,knights,0,8,40,14,6,25,8,5,6,Knights are one of the best attackers.
4,9,Human,monster,gryphon,gryphons,0,1,250,75,50,100,50,50,25,Gryphons are incredibly powerful monsters.
